In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,ShuffleSplit,GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn import model_selection

In [2]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[["League","Division","NewLeague"]])
y = df["Salary"]
X_ = df.drop(["Salary","League","Division","NewLeague"],axis = 1).astype("float64")
X = pd.concat([X_,dms[["League_N","Division_W","NewLeague_N"]]],axis = 1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25,random_state = 42)

In [6]:
from sklearn.ensemble import BaggingRegressor
bagged_model = BaggingRegressor(bootstrap_features = True)
bagged_model.fit(X_train,y_train)

BaggingRegressor(bootstrap_features=True)

In [7]:
bagged_model.n_estimators

10

In [9]:
bagged_model.estimators_

[DecisionTreeRegressor(random_state=1961753808),
 DecisionTreeRegressor(random_state=1262195226),
 DecisionTreeRegressor(random_state=1922057988),
 DecisionTreeRegressor(random_state=1343910323),
 DecisionTreeRegressor(random_state=861974212),
 DecisionTreeRegressor(random_state=2006230440),
 DecisionTreeRegressor(random_state=1689721198),
 DecisionTreeRegressor(random_state=531275470),
 DecisionTreeRegressor(random_state=872344952),
 DecisionTreeRegressor(random_state=703190317)]

In [10]:
bagged_model.estimators_samples_

[array([156,  35,  34,  16,  33,  44, 147, 137, 107, 115,  15,  85, 137,
        147,  46, 179, 137,  78,  39, 147,   5, 151,  64, 182, 114,  98,
         65, 130, 100, 162, 170, 146,  89, 158, 182,  77, 156, 171, 144,
         50, 173, 143, 120, 192, 140, 163,  92, 166,  24, 122,  87, 178,
         81,  59,  43,  31,  30,  15,  83,  11,  78,  47,   3,  21,  41,
          2,  85,  80, 154, 147,  45,  53, 173, 136, 170,   4, 144, 184,
        192, 152, 128,   8,  53,  14,  33,   2,  25, 144,  19, 142, 191,
        137,  75,  20, 108, 171, 127, 152, 127, 148, 133, 134,  88, 173,
         66, 150, 110,  67, 140,  26,  50,  81, 135, 166,  87, 107,  47,
        134, 125, 165,  80, 182, 100, 107,  68,  36,   5, 144,  18,  48,
         17,  94,  16, 118, 137, 187,  53,  31, 177, 119,  50, 139,   3,
        155,  61,  85,  92,  63, 196, 190,  36, 196, 111, 148,  73, 158,
        107, 133, 129, 121,  90, 108,  22, 184,   6, 179, 147, 169, 114,
         84, 183,  45, 191, 137, 182, 153,  61,  32

In [11]:
bagged_model.estimators_features_

[array([16,  5,  9,  8, 14, 13, 12,  9, 10,  6,  2,  1,  9, 16,  5, 17, 11,
         7,  9]),
 array([13,  9,  1,  7, 14, 16,  8,  7, 10,  5, 13,  1,  7,  2, 16, 10,  9,
         0, 13]),
 array([ 4, 11,  5,  5,  3, 17,  2, 10,  5, 13,  0, 10, 18, 17, 10,  7,  1,
        10,  9]),
 array([10,  1,  0,  8,  3,  0, 17, 12, 10,  7,  9, 12, 18,  4, 11,  4, 17,
        13, 10]),
 array([ 4,  8,  6,  9,  6, 18,  1,  1,  4,  6, 16, 15, 17,  9, 18, 17, 16,
         9,  3]),
 array([ 8,  6, 14,  0,  0,  8, 10,  1,  6, 16,  0,  3,  1,  7, 11,  1,  5,
        10,  7]),
 array([14, 14,  7,  3,  4,  7, 12, 16,  8, 14, 10, 11, 15, 18, 16, 13, 15,
        15, 15]),
 array([14,  9,  9,  6,  1,  7, 18,  5, 16, 10,  8,  8,  8,  4, 13,  5,  6,
         6, 10]),
 array([ 7,  7,  9,  3, 18, 14,  0, 18, 15, 14, 11,  5,  3,  1,  7, 14,  9,
         5,  1]),
 array([13, 13, 10, 14, 18,  6,  0,  6,  1, 18, 15,  5,  9, 14, 17,  3,  4,
         8, 12])]

In [12]:
bagged_model.estimators_[0]

DecisionTreeRegressor(random_state=1961753808)

# Tahmin

In [13]:
y_pred = bagged_model.predict(X_test)

In [14]:
np.sqrt(mean_squared_error(y_test,y_pred))

351.1638455761843

In [15]:
iki_y_pred = bagged_model.estimators_[1].fit(X_train,y_train).predict(X_test)

In [16]:
np.sqrt(mean_squared_error(y_test,y_pred))

351.1638455761843

In [17]:
iki_y_pred = bagged_model.estimators_[6].fit(X_train,y_train).predict(X_test)

In [18]:
np.sqrt(mean_squared_error(y_test,y_pred))

351.1638455761843

# Model Tuning

In [19]:
bagged_model = BaggingRegressor(bootstrap_features = True)
bagged_model.fit(X_train,y_train)

BaggingRegressor(bootstrap_features=True)

In [21]:
bagged_params = {"n_estimators":range(2,20)}

In [22]:
bagged_cv_model = GridSearchCV(bagged_model,bagged_params,cv = 10)

In [24]:
bagged_cv_model.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=BaggingRegressor(bootstrap_features=True),
             param_grid={'n_estimators': range(2, 20)})

In [25]:
bagged_cv_model.best_params_

{'n_estimators': 16}

In [26]:
bagged_tuned = BaggingRegressor(n_estimators = 16,random_state = 45)

In [27]:
bagged_tuned.fit(X_train,y_train)

BaggingRegressor(n_estimators=16, random_state=45)

In [29]:
y_pred = bagged_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

343.8174114256221